In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Gerekli kütüphaneleri yükle

import torch
import json
from datasets import DatasetDict, Dataset

# CUDA kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılacak cihaz: {device}")

# Veri setlerini yükle
with open("/content/drive/MyDrive/llm_qa/electra/final_train_data_v2.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open("/content/drive/MyDrive/llm_qa/electra/final_dev_data_v2.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# Veri setini düzenleyen fonksiyon
def prepare_data(data):
    contexts, questions, answers = [], [], []
    for item in data['data']:
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                contexts.append(context)
                questions.append(qa['question'])
                answers.append({
                    'text': qa['answers'][0]['text'],
                    'answer_start': qa['answers'][0]['answer_start']
                })
    return contexts, questions, answers

# Eğitim ve doğrulama verilerini hazırlayın
train_contexts, train_questions, train_answers = prepare_data(train_data)
dev_contexts, dev_questions, dev_answers = prepare_data(dev_data)

# Hugging Face Dataset formatına dönüştürme
train_dataset = Dataset.from_dict({
    "context": train_contexts,
    "question": train_questions,
    "answers": train_answers
})
dev_dataset = Dataset.from_dict({
    "context": dev_contexts,
    "question": dev_questions,
    "answers": dev_answers
})

dataset = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset
})


Kullanılacak cihaz: cuda


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Model ve Tokenizer
model_name = "dbmdz/electra-base-turkish-cased-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at dbmdz/electra-base-turkish-cased-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_data(examples):

     tokenized_examples = tokenizer(
         examples["question"],
         examples["context"],
         max_length=384,
         truncation=True,
         padding="max_length",
         return_offsets_mapping=True
     )

     start_positions = []
     end_positions = []

     for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
         answer = examples["answers"][i]
         start_char = answer["answer_start"]
         end_char = start_char + len(answer["text"])

         # Offset içindeki token başlangıç ve bitiş pozisyonlarını bulma
         start_token = None
         end_token = None
         for j, (start, end) in enumerate(offsets):
             if start <= start_char < end:
                 start_token = j
             if start < end_char <= end:
                 end_token = j

         # Pozisyonları ayarlama
         if start_token is None or end_token is None:
             start_token, end_token = 0, 0

         start_positions.append(start_token)
         end_positions.append(end_token)

     tokenized_examples["start_positions"] = start_positions
     tokenized_examples["end_positions"] = end_positions
     return tokenized_examples

# Veri setini işleme
tokenized_datasets = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=["context", "question", "answers"]
)


Map:   0%|          | 0/14221 [00:00<?, ? examples/s]

Map:   0%|          | 0/1330 [00:00<?, ? examples/s]

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 430574 KiB | 430574 KiB | 430574 KiB |      0 B   |
|       from large pool | 430080 KiB | 430080 KiB | 430080 KiB |      0 B   |
|       from small pool |    494 KiB |    494 KiB |    494 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 430574 KiB | 430574 KiB | 430574 KiB |      0 B   |
|       from large pool | 430080 KiB | 430080 KiB | 430080 KiB |

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Eğitim parametrelerini ayarla
output_dir = 'electra/results'

training_args = TrainingArguments(
    output_dir=output_dir,          # Model çıktı dosyalarının kaydedileceği klasör
    evaluation_strategy="steps",    # Değerlendirme sıklığı (adım bazlı)
    eval_steps=500,                 # Kaç adımda bir değerlendirme yapılacağı
    learning_rate=5e-5,             # Öğrenme oranı
    per_device_train_batch_size=16, # Her cihaz için eğitim batch boyutu
    per_device_eval_batch_size=16,  # Her cihaz için değerlendirme batch boyutu
    num_train_epochs=9,             # Eğitim boyunca epoch sayısı
    weight_decay=0.01,              # Ağırlık azaltma (regularization)
    save_steps=500,                 # Model kaydedilecek adım sayısı
    save_total_limit=2,             # Maksimum kaç model kaydedileceği
    logging_dir="electra/logs",  # Log dosyalarının kaydedileceği klasör
    logging_steps=100,              # Kaç adımda bir log kaydedileceği
    load_best_model_at_end=True,    # En iyi modeli eğitim sonunda yükler
    push_to_hub=False               # Hugging Face Hub'a yükleme seçeneği (opsiyonel)
)

# Trainer nesnesini tanımlayın
trainer = Trainer(
    model=model,                          # Kullanılacak model
    args=training_args,                   # Eğitim parametreleri
    train_dataset=tokenized_datasets["train"],    # Eğitim veri seti
    eval_dataset=tokenized_datasets["validation"], # Doğrulama veri seti
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early Stopping eklendi

)

# Model eğitimi
trainer.train()

print("Model eğitimi tamamlandı!")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,1.143500,1.308736
1000,0.786800,1.197267
1500,0.787100,1.180289
2000,0.484100,1.471283
2500,0.477200,1.284430
3000,0.346600,1.409693


Model eğitimi tamamlandı!


In [ ]:
!mv /electra /content/drive/MyDrive/llm_qa/electra
!mv /wandb /content/drive/MyDrive/llm_qa/wandb


mv: cannot stat '/electra': No such file or directory
mv: cannot stat '/wandb': No such file or directory


In [ ]:
!mv /content/wandb /content/drive/MyDrive/llm_qa/electra
!mv /content/electra /content/drive/MyDrive/llm_qa/wandb

In [16]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Checkpoint yolunu belirtin
checkpoint_path = "/content/drive/MyDrive/llm_qa/wandb/results/checkpoint-1500"
model_name = "loodos/albert-base-turkish-uncased"


# Tokenizer ve modeli yükle
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Checkpoint'teki tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Test verisi
context = """Feyzullah Efendi Vakası, 1703 yılında İstanbul'da başlayan, II. Mustafa ile hocası ve yakın danışmanı Şeyhülislam Feyzullah Efendi aleyhine gelişen büyük bir ayaklanmadır. Sultan II. Mustafa'nın Erzurum'dan getirterek şeyhülislamlığa yükselttiği hocası Feyzullah Efendi, ayaklanmanın sebebi olduğu için olay, Feyzullah Efendi'nin ismiyle de anılmaya başladı. Seyyid Feyzullah Efendi ikinci defa getirildiği şeyhülislâmlık görevinde devlet işlerine müdahale etmesi, ayaklanmanın ortaya çıkmasının ana sebebidir. Feyzullah Efendi'nin devlet işlerine karışarak terfi, tayin ve azil işlerinde aldırtdığı kararlar nedeniyle saray çevresi, asker ve ulemada rahatsızlık oluşturmaya başlamıştır. Bunun yanı sıra II. Mustafa'nın Edirne'de devlet işleriyle ilgilenmeyip avcılık ile uğraşarak devlet idaresinden uzak kalması, uzun süredir yaşanan ekonomik kriz ile devletin başkentinin Edirne'ye taşınacağı iddiaları da İstanbul halkının isyana katılma nedenlerini oluşturmuştur. 17 Temmuz 1703 tarihinde Şeyhülislam Feyzullah Efendi'nin etkinliğinden rahatsız olan bazı devlet erkanının yönlendirmesiyle gecikmiş ulufeleri bahane eden cebeciler, ayaklanma başlattı. Kısa süre sonra Feyzullah Efendi Vakası; yeniçerilerin, medrese talebelerinin ve İstanbul halkının katılımıyla büyük bir isyana dönüştü. İstanbul'da denetimi ele geçiren isyancılar, sadrazamlığa Kavanoz Ahmed Paşa'yı, şeyhülislamlığa da İmam Mehmed Efendi'yi getirdiler. Bir süre sonra isyancılar isteklerini bildirmek üzere Edirne'ye bir heyet gönderdiler. Ancak isyancıların isteklerinin bildirildiği heyet, Feyzullah Efendi'nin emriyle Havsa civarında tutuklandı. II. Mustafa, isyancıları yumuşatmak amacıyla 27 Temmuz 1703 tarihinde Feyzullah Efendi'yi şeyhülislamlık görevinden aldı. Ancak isyancılar, asker ve halktan oluşan yaklaşık 60.000 kişilik bir kuvvetle ağustos ayı başlarında Edirne'ye harekete geçti. Bu ihtilal ordusu Silivri'ye geldiklerinde II. Mustafa'nın tahttan indirilip öz kardeşi Ahmed'in tahta geçirilmesi için bir karar ve fetva aldılar. İhtilal kuvvetleri engellenemedi ve 20 Ağustos'ta ihtilal kuvveti Edirne'ye ulaştı. 22 Ağustos'ta II. Mustafa tahttan çekilerek yerini öz kardeşi III. Ahmet'e bıraktı. Yeni padişahın onayı alan asiler 3 Eylül'de ağır işkence yaptıkları Feyzullah Efendi'yi öldürdüler. 4 Eylül 1703'de Sultan III. Ahmet İstanbul'a hareket etti. Böylece Edirne'nin fiilen Osmanlı devletine başkentlik etmesinin ikinci dönemi kapanmış oldu. 14 Eylül'de İstanbul'a ulaşan III. Ahmet giderek daha şiddetli ve sert tedbirler alarak devlet idaresini isyancıların elinden kurtarmaya başladı. Ancak 1704'ün ilk aylarında Padişah III. Ahmet egemen olabildi."""

question = "Seyyid Feyzullah Efendi ikinci defa getirildiği şeyhülislâmlık görevinde devlet işlerine müdahale etmesi, neyin ana sebebidir?"

# Soruyu ve bağlamı birleştir ve tokenleştir
inputs = tokenizer(
    question,
    context,
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding="max_length"
).to("cuda")

# Modelden çıkış al
with torch.no_grad():
    outputs = model(**inputs)

# Başlangıç ve bitiş pozisyonlarını alın
start_index = outputs.start_logits.argmax(dim=-1).item()
end_index = outputs.end_logits.argmax(dim=-1).item()

# Cevabı çözümleme
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index : end_index + 1])
)

# Çıktıyı yazdır
print(f"Soru: {question}")
print(f"Cevap: {answer}")

Soru: Seyyid Feyzullah Efendi ikinci defa getirildiği şeyhülislâmlık görevinde devlet işlerine müdahale etmesi, neyin ana sebebidir?
Cevap: lanmanın ortaya çıkmasının
